# Informe EDA

In [234]:
%run lib.ipynb

# Lectura del dataset de películas
df_movies = pd.read_parquet("datasets/movies_dataset.parquet")
df_movies.info()
df_movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45376 non-null  object 
 2   genres                 45376 non-null  object 
 3   id                     45376 non-null  object 
 4   original_language      45365 non-null  object 
 5   overview               44435 non-null  object 
 6   popularity             45376 non-null  float64
 7   production_companies   45376 non-null  object 
 8   production_countries   45376 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45376 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0,1995,12.451801
1,None,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000
3,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000


## Chequeo de nulos

In [235]:
#suma de valores nulos por columna
print(columnas_de_nulos(df_movies))


belongs_to_collection    40888
original_language           11
overview                   941
runtime                    246
status                      80
tagline                  24978
dtype: int64


### De un total de 45376, la columna "belongs_to_collection" tiene un 90.1% de valores nulos, como además la columna no será utilizada en el API (ni en el recomendador), será eliminada

In [236]:
df_movies = df_movies.drop(['belongs_to_collection'], axis=1)
print(columnas_de_nulos(df_movies))

original_language       11
overview               941
runtime                246
status                  80
tagline              24978
dtype: int64


### Para la columna "original_language", tenemos 11 filas con valor nulo. 8 de ellos tienen sólo un valor en la columna "spoken_languages" así que lo imputaremos en "original_languaje", para las 4 restantes imputaremos el primer valor que se encuentre en #spoken_languajes". Esta columna es de interes ya que posteriormente la utilizaré para reducir el dataset y poder factibilizar la ejecución del algoritmo de recomendación, ya que con todo el dataset genera problemas de memoria.

In [237]:
df_movies.loc[[21579, 22805, 32103], ['original_language']] = 'en'
df_movies.loc[[19559, 44494], ['original_language']] = ''
df_movies.loc[[37354], ['original_language']] = 'cs'
df_movies.loc[[40982], ['original_language']] = 'ur'
df_movies.loc[[41802, 44573], ['original_language']] = 'xx'
df_movies.loc[[43978], ['original_language']] = 'fr'
df_movies.loc[[44328], ['original_language']] = 'sv'

print(columnas_de_nulos(df_movies))


overview      941
runtime       246
status         80
tagline     24978
dtype: int64


### Utilizaré la columna "overview" como la base del algoritmo de recomendación, por lo que necesito que este poblada para todas las películas. Utilizaré en primera instancia la columna "tagline" y posteriormente la columna "title".

In [238]:

#Reemplazo los valores nulos de la columna overview por su correspondiente en tagline
df_movies["overview"] = df_movies["overview"].fillna(df_movies["tagline"])

#Reemplazo los valores nulos de la columna overview por su correspondiente en title
df_movies["overview"] = df_movies["overview"].fillna(df_movies["title"])

print(columnas_de_nulos(df_movies))


runtime      246
status        80
tagline    24978
dtype: int64


### La columna "runtime" es de interes para generar correlaciones posteriormente, voy a utilizar el valor promedio para rellenar los valores nulos ya que representan sólo el 0.51% de los datos en esa columna. 

In [239]:
df_movies["runtime"] = df_movies["runtime"].fillna(df_movies["runtime"].mean())
print(columnas_de_nulos(df_movies))


status        80
tagline    24978
dtype: int64


### Todos los registros con valor nulo en la columna "status", presentan "release_year" en fechas pasadas y además indicadores en la columna "popularity", por loq ue asignaremos el valor "Released" para todos los registros con valor nulo.

In [240]:
df_movies[df_movies['status'].isnull()]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
189,0,"[{'id': 99, 'name': 'Documentary'}]",56088,en,A documentary about the culture of hip-hop. Th...,0.159596,[],[],1995-08-25,0.0,93.000000,[],None,What Happens On Stage Is Only Half The Story!,The Show,5.3,2.0,1995,0.0
765,0,"[{'id': 99, 'name': 'Documentary'}]",52059,en,The Gate of Heavenly Peace is a feature-length...,0.107050,[],[],1995-01-01,0.0,180.000000,[],None,None,The Gate of Heavenly Peace,6.7,3.0,1995,0.0
1382,0,"[{'id': 35, 'name': 'Comedy'}]",40506,en,Wally Sparks is a tabloid TV show reporter who...,0.576741,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1997-01-01,0.0,105.000000,"[{'iso_639_1': 'en', 'name': 'English'}]",None,None,Meet Wally Sparks,5.3,10.0,1997,0.0
2138,0,[],77314,fr,The Cabinet of Dr. Ramirez,0.037500,[],[],1991-12-04,0.0,94.181675,[],None,None,The Cabinet of Dr. Ramirez,0.0,0.0,1991,0.0
4126,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",109472,en,The Girl,0.001653,[],[],2001-06-06,0.0,0.000000,[],None,None,The Girl,0.0,0.0,2001,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44295,0,[],39827,en,A celebration of 30 years of The Who.,0.014260,[],[],1994-08-15,0.0,155.000000,[],None,None,The Who: Thirty Years of Maximum R&B,0.0,0.0,1994,0.0
44983,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",49492,en,"Jogwa actually means alms given to a person, u...",0.004990,[],[],2009-09-25,0.0,92.000000,[],None,The Awakening,Jogwa - The Awakening,0.0,0.0,2009,0.0
45005,0,[],128268,en,Documentary about Veruschka von Lehndorff.,0.056298,[],"[{'iso_3166_1': 'DE', 'name': 'Germany'}]",2005-09-30,0.0,52.000000,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",None,None,Veruschka: A Life for the Camera,6.0,1.0,2005,0.0
45073,0,[],41811,en,"At a spacious house in Los Angeles, Helene is ...",0.000002,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1990-11-30,0.0,110.000000,"[{'iso_639_1': 'en', 'name': 'English'}]",None,None,Eating,0.0,0.0,1990,0.0


In [241]:
df_movies['status'] = df_movies['status'].fillna('Released')
print(columnas_de_nulos(df_movies))

tagline    24978
dtype: int64


### La columna "tagline" presenta un 50% de valores nulos, así que sólo le asignaremos ''.

In [242]:
df_movies["tagline"] = df_movies["tagline"].fillna('')
print(columnas_de_nulos(df_movies))
print('\n')
df_movies.info()

Series([], dtype: float64)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45376 non-null  object 
 1   genres                45376 non-null  object 
 2   id                    45376 non-null  object 
 3   original_language     45376 non-null  object 
 4   overview              45376 non-null  object 
 5   popularity            45376 non-null  float64
 6   production_companies  45376 non-null  object 
 7   production_countries  45376 non-null  object 
 8   release_date          45376 non-null  object 
 9   revenue               45376 non-null  float64
 10  runtime               45376 non-null  float64
 11  spoken_languages      45376 non-null  object 
 12  status                45376 non-null  object 
 13  tagline               45376 non-null  object 
 14  title                 45376 non-null  obj

## Valores duplicados

### Buscaremos valores duplicados para filas completas

In [243]:
df_movies[df_movies.duplicated(keep=False)]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
676,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",105045,de,"East-Berlin, 1961, shortly after the erection ...",0.122178,"[{'name': 'Studio Babelsberg', 'id': 264}, {'n...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,5.0,1.0,1995,0.0
1463,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",105045,de,"East-Berlin, 1961, shortly after the erection ...",0.122178,"[{'name': 'Studio Babelsberg', 'id': 264}, {'n...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,5.0,1.0,1995,0.0
7340,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",5511,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,"[{'name': 'Fida cinematografica', 'id': 73}, {...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1967-10-25,39481.0,105.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0,1967,0.0
8063,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",23305,en,"In feudal India, a warrior (Khan) who renounce...",1.967992,"[{'name': 'Filmfour', 'id': 6705}]","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",2001-09-23,0.0,86.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,,The Warrior,6.3,15.0,2001,0.0
9160,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",5511,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,"[{'name': 'Fida cinematografica', 'id': 73}, {...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1967-10-25,39481.0,105.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0,1967,0.0
9322,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",23305,en,"In feudal India, a warrior (Khan) who renounce...",1.967992,"[{'name': 'Filmfour', 'id': 6705}]","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",2001-09-23,0.0,86.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,,The Warrior,6.3,15.0,2001,0.0
13215,0,"[{'id': 99, 'name': 'Documentary'}]",159849,en,"The third film of Frank Capra's 'Why We Fight""...",0.473322,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1943-01-01,0.0,57.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Why We Fight: Divide and Conquer,5.0,1.0,1943,0.0
13256,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",141971,fi,Recovering from a nail gun shot to the head an...,0.411949,"[{'name': 'Filmiteollisuus Fine', 'id': 5166}]","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",2008-12-26,0.0,108.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,Which one is the first to return - memory or t...,Blackout,6.7,3.0,2008,0.0
13370,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",141971,fi,Recovering from a nail gun shot to the head an...,0.411949,"[{'name': 'Filmiteollisuus Fine', 'id': 5166}]","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",2008-12-26,0.0,108.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,Which one is the first to return - memory or t...,Blackout,6.7,3.0,2008,0.0
13598,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",168538,en,"In Zola's Paris, an ingenue arrives at a tony ...",1.276602,"[{'name': 'Cannon Group', 'id': 1444}, {'name'...",[],1983-06-13,0.0,92.0,[],Released,,"Nana, the True Key of Pleasure",4.7,3.0,1983,0.0


### Eliminaremos las duplicadas quedándonos con la primera ocurrencia

In [244]:
df_movies = df_movies.drop_duplicates(keep='first')
df_movies[df_movies.duplicated(keep=False)]


,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return


### Buscaremos valores duplicados sólo para la columna id

In [245]:
df_movies[df_movies.duplicated(keep=False, subset=['id'])].sort_values(by=['id','popularity'])

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
44738,16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",10991,ja,When Molly Hale's sadness of her father's disa...,6.480376,"[{'name': 'TV Tokyo', 'id': 3034}, {'name': '4...","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",2000-07-08,68411275.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,144.0,2000,4.275705
4110,16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",10991,ja,When Molly Hale's sadness of her father's disa...,10.264597,"[{'name': 'TV Tokyo', 'id': 3034}, {'name': '4...","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",2000-07-08,68411275.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,143.0,2000,4.275705
20877,0,"[{'id': 18, 'name': 'Drama'}]",109962,en,Two literary women compete for 20 years: one w...,10.396878,"[{'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8...","[{'iso_3166_1': 'US', 'name': 'United States o...",1981-09-23,0.0,115.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"From the very beginning, they knew they'd be f...",Rich and Famous,4.9,7.0,1981,0.000000
5706,0,"[{'id': 18, 'name': 'Drama'}]",109962,en,Two literary women compete for 20 years: one w...,12.180836,"[{'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8...","[{'iso_3166_1': 'US', 'name': 'United States o...",1981-09-23,0.0,115.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"From the very beginning, they knew they'd be f...",Rich and Famous,4.9,7.0,1981,0.000000
23507,3512454,"[{'id': 18, 'name': 'Drama'}]",110428,fr,"Winter, 1915. Confined by her family to an asy...",0.110065,"[{'name': 'Canal+', 'id': 5358}, {'name': 'Art...","[{'iso_3166_1': 'FR', 'name': 'France'}]",2013-03-13,115860.0,95.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,,Camille Claudel 1915,7.0,20.0,2013,0.032985
4352,3512454,"[{'id': 18, 'name': 'Drama'}]",110428,fr,"Winter, 1915. Confined by her family to an asy...",0.134014,"[{'name': 'Canal+', 'id': 5358}, {'name': 'Art...","[{'iso_3166_1': 'FR', 'name': 'France'}]",2013-03-13,115860.0,95.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,,Camille Claudel 1915,7.0,20.0,2013,0.032985
44743,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",12600,ja,"All your favorite Pokémon characters are back,...",6.080108,[],"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2001-07-06,28023563.0,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0,2001,0.000000
5531,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",12600,ja,"All your favorite Pokémon characters are back,...",7.072301,[],"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2001-07-06,28023563.0,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0,2001,0.000000
11337,2500,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",13209,fa,"Since women are banned from soccer matches, Ir...",1.528960,"[{'name': 'Jafar Panahi Film Productions', 'id...","[{'iso_3166_1': 'IR', 'name': 'Iran'}]",2006-05-26,0.0,93.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,,Offside,6.7,27.0,2006,0.000000
15758,2500,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",13209,fa,"Since women are banned from soccer matches, Ir...",1.529879,"[{'name': 'Jafar Panahi Film Productions', 'id...","[{'iso_3166_1': 'IR', 'name': 'Iran'}]",2006-05-26,0.0,93.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,,Offside,6.7,27.0,2006,0.000000


### Existen valores duplicados en la columna "id", la diferencia al analizar los registros se concentra en la columna "popularity", eliminaremos los registros duplicados conservando el que tenga la mayor popularidad.

In [ ]:
df_movies = df_movies.sort_values(by=['id','popularity'], kind='mergesort').drop_duplicated('id', keep='last')
#df_movies[df_movies.duplicated(keep=False, subset=['id'])].sort_values(by=['id','popularity'])
df_movies.info()

/tmp/ipykernel_10204/1345200008.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_movies.sort_values(by=['id','popularity'])[df_movies.duplicated(keep="first", subset=['id'])].drop()


ValueError: Need to specify at least one of 'labels', 'index' or 'columns'

## Valores faltantes

### Realizaremos el análisis para las columnas relevantes para el funcionamiento de los endpoints de la API. Estos son las columnas "id", "original_language", "popularity", "title", "vote_average", "vote_count", "return", "overview" y "release_date".

In [ ]:
selected_columns = ['id','original_language','title','release_date','return','vote_count','vote_average','popularity','overview']
print(df_movies[selected_columns].isna().sum())


id                   0
original_language    0
title                0
release_date         0
return               0
vote_count           0
vote_average         0
popularity           0
overview             0
dtype: int64


### No se detectan valores faltantes.

In [ ]:
selected_columns = ['id','original_language','title','release_date','return','vote_count','vote_average','popularity','overview']
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 44738 to 21094
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                26 non-null     object 
 1   genres                26 non-null     object 
 2   id                    26 non-null     object 
 3   original_language     26 non-null     object 
 4   overview              26 non-null     object 
 5   popularity            26 non-null     float64
 6   production_companies  26 non-null     object 
 7   production_countries  26 non-null     object 
 8   release_date          26 non-null     object 
 9   revenue               26 non-null     float64
 10  runtime               26 non-null     float64
 11  spoken_languages      26 non-null     object 
 12  status                26 non-null     object 
 13  tagline               26 non-null     object 
 14  title                 26 non-null     object 
 15  vote_average          2

## Tipos de datos

### se analizarán los tipos de datos para las columnas relevantes.

## Análisis estadístico

## Análisis de correlaciones